In [51]:
import pandas as pd
import json

from pandas import json_normalize



In [43]:
import pandas as pd

df = pd.read_json('/content/DataEngineeringQ2.json')


In [45]:
#df = df.explode(['patientDetails', 'consultationData.medicines'])


In [36]:
with open('/content/DataEngineeringQ2.json', 'r') as file:
    data = json.load(file)

patient_details = [entry['patientDetails'] for entry in data]
df = pd.DataFrame(patient_details)

In [57]:
patient_details = [entry['patientDetails'] for entry in data]
consultation_data = [entry['consultationData'] for entry in data]

In [58]:
df_patient_details = pd.DataFrame(patient_details)
df_consultation_data = pd.DataFrame(consultation_data)

In [59]:
df_patient_details['gender'].replace('', pd.NA, inplace=True)
mode_gender = df_patient_details['gender'].mode()[0]
df_patient_details['gender'].fillna(mode_gender, inplace=True)

In [25]:
df['gender'].replace('', pd.NA, inplace=True)
mode_gender = df['gender'].mode()[0]
df['gender'].fillna(mode_gender, inplace=True)


In [26]:
df.head()

,_id,firstName,lastName,emailId,gender,alternateContact,birthDate,phrId,chat,customId
0,T6hb630b3,Css,,,M,,None,NaN,NaN,NaN
1,T6h33b300,Lokesh,,,M,NaN,1996-05-16T18:30:00.000Z,NaN,NaN,NaN
2,TjhB4373,Shila,Das,,M,NaN,NaN,63b5hvy614d5,NaN,NaN
3,6df4R5b,Bhavika,Ben Panchal,,F,NaN,1988-04-24T14:30:00.000Z,644nig7y,NaN,NaN
4,lK9hy06a,Raghu Viju,,,M,,NaN,NaN,NaN,NaN


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               31 non-null     object
 1   firstName         31 non-null     object
 2   lastName          31 non-null     object
 3   emailId           30 non-null     object
 4   gender            31 non-null     object
 5   alternateContact  9 non-null      object
 6   birthDate         21 non-null     object
 7   phrId             8 non-null      object
 8   chat              16 non-null     object
 9   customId          16 non-null     object
dtypes: object(10)
memory usage: 2.5+ KB


In [28]:
print(df)

                       _id                 firstName     lastName emailId  \
0                T6hb630b3                       Css                        
1                T6h33b300                    Lokesh                        
2                 TjhB4373                     Shila          Das           
3                  6df4R5b                   Bhavika  Ben Panchal           
4                 lK9hy06a                Raghu Viju                        
5                  kI9d5c6              Dinesh Kumar                        
6                 i9R324b7                     Lalit     Sankhwal           
7   T6h41b5ad3d8002ad9c3c4                      Ravi                        
8                 T9Iy347e                    Dinesh                        
9                  T4Rb1a2                    Sanjay                        
10                Tr4T2173                    Akshay       Akshay           
11                  T6Wa50                   Nirmala                        

In [29]:
female_percentage = (df['gender'] == 'female').mean() * 100
female_percentage = round(female_percentage, 2)
print(female_percentage)


0.0


In [30]:
from datetime import datetime

df['birthDate'] = pd.to_datetime(df['birthDate'], errors='coerce')
current_date = datetime.now()

df['age'] = df['birthDate'].apply(lambda x: current_date.year - x.year if pd.notnull(x) else None)


In [31]:
def categorize_age(age):
    if age is None:
        return 'Unknown'
    elif age <= 12:
        return 'Child'
    elif age <= 19:
        return 'Teen'
    elif age <= 59:
        return 'Adult'
    else:
        return 'Senior'

df['ageGroup'] = df['age'].apply(categorize_age)


In [32]:
adult_count = df[df['ageGroup'] == 'Adult'].shape[0]
print(adult_count)


21


In [33]:

consultation_data = [entry['consultationData'] for entry in data]

df_patient_details = pd.DataFrame(patient_details)
df_consultation_data = pd.DataFrame(consultation_data)


In [34]:
df_consultation_data['num_medicines'] = df_consultation_data['medicines'].apply(lambda x: len(x))


In [35]:
average_medicines = df_consultation_data['num_medicines'].mean()
average_medicines = round(average_medicines, 2)
print(average_medicines)


2.13


In [61]:
df_consultation_data = df_consultation_data.explode('medicines', ignore_index=True)
df_medicines = json_normalize(df_consultation_data['medicines'])


In [68]:
df_medicines = pd.DataFrame({
    'phoneNumber': ['+919876787687', '919877475896', '+913454768688', '9876787687', '3454768688', '57769666']
})

def is_valid_indian_phone_number(phone_number):
    phone_number = phone_number.strip()

    if phone_number.startswith('+91'):
        phone_number = phone_number[3:]
    elif phone_number.startswith('91'):
        phone_number = phone_number[2:]

    if len(phone_number) == 10 and phone_number.isdigit():
        number = int(phone_number)
        return 6000000000 <= number <= 9999999999
    else:
        return False

df_medicines['isValidMobile'] = df_medicines['phoneNumber'].apply(is_valid_indian_phone_number)

valid_phone_count = df_medicines['isValidMobile'].sum()
print(f"Number of valid phone numbers found: {valid_phone_count}")

print(df_medicines)


Number of valid phone numbers found: 3
     phoneNumber  isValidMobile
0  +919876787687           True
1   919877475896           True
2  +913454768688          False
3     9876787687           True
4     3454768688          False
5       57769666          False


In [74]:
df_consultation_data = df_consultation_data.explode('medicines', ignore_index=True)
df_medicines = json_normalize(df_consultation_data['medicines'])

df_consultation_data['num_medicines'] = df_consultation_data['medicines'].apply(lambda x: len(x) if isinstance(x, list) else 0)

df_combined = df_patient_details.join(df_consultation_data[['num_medicines']], how='inner')

df_combined['birthDate'] = pd.to_datetime(df_combined['birthDate'], errors='coerce')
current_date = datetime.now()
df_combined['age'] = df_combined['birthDate'].apply(lambda x: current_date.year - x.year if pd.notnull(x) else None)

df_combined = df_combined.dropna(subset=['age', 'num_medicines'])

correlation = df_combined[['age', 'num_medicines']].corr().loc['age', 'num_medicines']
correlation_rounded = round(correlation, 2)



In [75]:
print(correlation_rounded)


nan


In [77]:
df_consultation_data = pd.DataFrame(consultation_data)

df_consultation_data = df_consultation_data.explode('medicines', ignore_index=True)
df_medicines = json_normalize(df_consultation_data['medicines'])

if 'status' in df_medicines.columns:
    df_medicines.dropna(subset=['status'], inplace=True)

    status_counts = df_medicines['status'].value_counts()

    total_medicines = status_counts.sum()
    percentage_distribution = (status_counts / total_medicines) * 100

    percentage_distribution = percentage_distribution.round(2)

    print("\nTotal active and inactive medicines:")
    print(status_counts)
    print("\nPercentage distribution of active and inactive medicines:")
    print(percentage_distribution)
else:
    print("'status' key not found in medicines data.")


'status' key not found in medicines data.
